# Quantum wavefunctions as restricted Boltzmann machines (RBM)
(This neural network ansatz was introduced in paper [Solving the quantum many-body problem with artificial neural networks](http://doi.org/10.1126/science.aag2302).)

The generic $N$-spin wavefunction in the Ising basis is 
$$
\lvert\psi\rangle = \sum_{\{\sigma\}}\Psi(\sigma_1, \dots, \sigma_N) \lvert\sigma_1, \dots, \sigma_N \rangle
$$
The RBM ansatz defines a set of $M$ hidden binary variable $h_j$ and the wavefunction elements are given based on a set of complex-valued weights $w_{ij}$, between the hidden and visible spins, and corresponding biases (local fields) $a_i$ and $b_j$ for the visible and hidden spins respecitively.
$$
    \Psi(\{\sigma\}, \{h\}) = \frac{1}{Z} \exp{\left(\sum_i a_i \sigma_i + \sum_{ij} w_{ij}\sigma_ih_j + \sum_j b_j h_j\right)}
$$
This means we are charactrising a linear map from a set of $2^N$ spin configurations to a complex number by $N\times M+N+M$ variables.

For a given set of weights $w'$ and biases $b'$ integrating the the hidden layer variables we get
$$
    \Psi(\{\sigma\}) = \frac{1}{Z} \exp{\left(\sum_i a_i \sigma_i\right)} \times  \prod_j 2\cosh(\theta_j) \quad \text{where} \quad
    \theta_j = \sum_{i} w'_{ij}\sigma_i + b'_j
$$

Note that the variable $Z$ is just introduced for normalization (we will ignore it throughtout because we explicitely normalized the wavefunctions and expectation values).

In [ ]:
using LinearAlgebra
using QuadGK

In [ ]:
"Restricted Boltzmann machine quantum state Ansatz"
mutable struct QRBMState
    N  :: Int
    M  :: Int
    a  :: Vector{ComplexF64}
    b  :: Vector{ComplexF64}
    w  :: Matrix{ComplexF64}
    σs :: Vector{Int}
    θs :: Vector{ComplexF64}
    amp :: ComplexF64
end

function QRBMState(N::Int, M::Int)
    scale = 0.1
    w   = scale * randn(ComplexF64, N, M)
    a   = randn(ComplexF64, N)
    b   = randn(ComplexF64, M)
    σs  = rand([1, -1], N)
    θs  = transpose(w)*σs + b
    amp = amplitude_rbmwf(σs, a, θs)
    QRBMState(N, M, a, b, w, σs, θs, amp)
end

function recalc_θs!(state::QRBMState)
    state.θs = transpose(state.w)*state.σs + state.b
end

function recalc_amp!(state::QRBMState)
    state.amp = amplitude_rbmwf(state.σs, state.a, state.θs)
end
#amplitude_rmbwf(s, a, θs) = exp(dot(s, a)) * prod(cosh.(θs))
amplitude_rbmwf(σs::Vector{Int}, a::Vector{ComplexF64}, θs::Vector{ComplexF64}) = 
    exp(dot(σs, a) + sum(log.(cosh.(θs))))

### Parameters
# `N` is the number of spins, `M` the number of hidden spins and `α=M/N` the ratio of hidden layer to visible layer
N = 16
α = 4
M = floor(α * N)
n_pars = N*M + N + M

state = QRBMState(N, M)
@show state.amp
@show state.σs;

## The optimization algorithm
### Gradient descent
We would like to represent the ground state of some Hamiltonian $H$ (for example the quantum Ising in transverse field) so we choose to minimize the variational energy with respect to $\psi$ that is a function of the current weights $w$ and biases $a, b$.
$$
    E = \frac{\langle \psi | H | \psi \rangle}{\langle \psi | \psi \rangle}
$$
The generalized force (the gradient of the variational energy) is a real-valued function of the complex weights and biases, therefore
$$
    F_k(w) = \frac{\partial}{\partial w_k} E(w) = 2 \frac{\partial}{\partial w^*_k}E(w, w^*) =
    \sum_{\boldsymbol{\sigma}} 
    \frac{\left[\frac{\partial}{\partial w^*_k} \psi^*(\boldsymbol{\sigma} )\right]\langle \boldsymbol{\sigma}  | H | \psi \rangle}{\langle \psi | \psi \rangle} - 
    \sum_\boldsymbol{\sigma}
    \frac{\psi^*(\boldsymbol{\sigma} )\langle \boldsymbol{\sigma}  | H | \psi \rangle}{\langle \psi | \psi \rangle} 
    \sum_{\boldsymbol{\sigma} '} 
    \frac{\left[\frac{\partial}{\partial w^*_k} \psi^*(\boldsymbol{\sigma} ')\right]\psi(\boldsymbol{\sigma} ')}{\langle \psi | \psi \rangle}
$$
As usual Interpreting $p(\boldsymbol{\sigma} ) = \frac{|\psi(\boldsymbol{\sigma} )|^2}{\langle \psi | \psi \rangle}$ as probability of configuration $\boldsymbol{\sigma} $ and defining the average (with some abuse of notation)
$\langle A\rangle = \sum_{\boldsymbol{\sigma}} p(\boldsymbol{\sigma}) A(\boldsymbol{\sigma})$ we have 
$$
F_k(w) =  \langle O^*_k \mathcal{E}\rangle - \langle O^*_k\rangle\langle \mathcal{E}\rangle
$$
where the operators $O_k(\boldsymbol{\sigma})$ and $\mathcal{E}(\boldsymbol{\sigma})$ are defined as 
$$
    \begin{aligned}
        \mathcal{E}(\boldsymbol{\sigma} ) & = \frac{\langle \boldsymbol{\sigma}  | H | \psi \rangle}{\psi(\boldsymbol{\sigma})}  \\
        O_k(\boldsymbol{\sigma}) & = \frac{\partial_k \psi(\boldsymbol{\sigma})}{\psi(\boldsymbol{\sigma})} 
    \end{aligned}
$$
The usual gradient descent algorithm can be implemented to update the weights and biases with learning rate $\lambda$ according to 
$$
    w_k^{i+1} = w_k^i - \lambda F_k(w^i)
$$
If we take the "generalized force" name seriously, a more physical update is to use the force to change the velocity and use that to change the weights. This approach is the momentum based gradient descent where with an initial velocity of $v^0 = 0$ and the momentum memeory amplitude is typically chosen to be $\gamma = 0.9$. Note that in physical terms $1-\gamma$ can be interpreted as friction amplitude.
$$
    \begin{aligned}
        v^i & = \gamma v^{i-1} + \lambda F_k(w^i) \\
        w_k^{i+1} &= w_k^i - v^i
    \end{aligned}
$$
While many different approached for the calculation of $v^{i+1}$ from $v^i$ exists, it turns out a simple calculation of the gradient in future (like in the backward Euler method) results in better convergence.
$$
    v^i  = \gamma v^{i-1} + \lambda F_k(w^i - \gamma v^{i-1})
$$

The derivatives can be taken explicitely. Based on the definition of $\psi(\boldsymbol{\sigma})$, the derivatives are given by
$$
    \begin{aligned}
        \partial_{a_i} \ln \psi &= \sigma_i \\
        \partial_{b_j} \ln \psi &= \tanh \theta_j \\
        \partial_{w_{ij}} \ln \psi &= \sigma_i \tanh \theta_j
    \end{aligned}
$$


### Monte Carlo evaluation
Since the configuration space is $2^N$ large, it is generally impossible to evaluate the above averages for problems of interest. Therefore, we should pick a sampling method to approximately evaluate the averages. At each step of the optimization, the Monte-Carlo sampling method picks a series of random configurations $\boldsymbol{\sigma}_1, \dots, \boldsymbol{\sigma}_\ell$ and evaluates the operators. New configurations are chosen by flipping a random spin and according to the acceptance probability 
$$
    P_{\text{accept}}(\boldsymbol{\sigma}_{\ell+1}) = \min \left(1, \left|\frac{\psi(\boldsymbol{\sigma}_{\ell+1})}{\psi(\boldsymbol{\sigma}_\ell)}\right|^2\right)
$$
To avoid recalculations, the set of $\theta_j$ is saved and updated only for the connection to the flipped spin.

Note that this process amount to a version of *stochastic* gradient descent as the total gradient is not computed at each step. The choice of configurations, however, is chosen by a Metropolis walk rather than uniform random to respect the probability distribution induced by the wavefunction over the configurations.

### Stochastic reconfiguration 
It is important to recognize that the wavefunction optimization (through its gradient) takes place in the space of weights. Therefore, instead of moving in the direction of the calculated gradient of the variational energy, one can choose to move in the direction suggested by the *power method*. In each step of the power method we apply the operator $\Lambda I - H$ to the wavefunction. For a large enough $\Lambda$ that insures the positivity of $\Lambda I-H$, the power method converges closer to the ground state
$$
    \psi^{t+1}_p = (\Lambda I - H) \psi^{t}
$$
In general the new wavefunction may not live entirely inside of the variational space, so we need to project it back to the variational space. Choosing a basis for the variational space
$$
    \psi^{t+1}_v = \psi^{t} + \sum_{k=1} \delta w_k ~\frac{\partial}{\partial w_k} \psi = \sum_{k=0}\delta w_k ~ O_k \psi
$$
We now require the projection of the power method wavefunction and variational change wavefunction to be the same. This gives a linear equation that solves for the direction of movements.
$$
    \langle \psi\rvert O^*_k (\Lambda -H) \lvert\psi\rangle = \sum_l \langle\psi \lvert O^*_k O_l \rvert\psi\rangle ~\delta w_l
$$
substituting $\Lambda$ from the equation for $k=0$ into the equation for general $k$ we find that
$$
    \langle  O^*_k \rangle\langle  H \rangle-\langle  O^*_k H \rangle = 
    \sum_l \left(\langle O^*_k O_l \rangle - \langle O^*_k \rangle\langle O_k \rangle\right) ~\delta w_l
$$
The solution is to move in the direction 
$$
    \delta w_l = - s^{-1}_{lk} F_k \qquad \text{where} \qquad s_{kl} = \langle O^*_k O_l \rangle - \langle O^*_k \rangle\langle O_k \rangle
$$
Note that this means we are moving in the direction of the gradient descend of the energy variation but with a new metric $s_{kl}$ !!!

In [ ]:
# Define the required Monte-Carlo functions
function propose_update!(state::QRBMState, spin::Int)
    σs_new = copy(state.σs)
    σs_new[spin] *= -1
    θs_new = state.θs + 2 * σs_new[spin] .* state.w[spin, :]
    new_amp = amplitude_rbmwf(σs_new, state.a, θs_new)
    if abs(new_amp/state.amp)^2 > rand()
        state.σs = σs_new
        state.θs = θs_new
        state.amp = new_amp
        return true
    end
    false
end

function measure(state::QRBMState, what::Symbol)
    N = state.N
    if what == :ZZ
        return [state.σs[i]*state.σs[mod1(i+1, N)] for i in 1:N]
    elseif what == :X
        sx = zeros(ComplexF64, N)
        for i = 1:N
            σi = copy(state.σs)
            σi[i] *= -1
            sx[i] =  amplitude_rbmwf(σi, state.a, state.θs + 2 * σi[i] .* state.w[i, :])/state.amp
        end
        return sx
    elseif what == :OStar
        partial_bs = tanh.(state.θs)
        return conj([state.σs...;partial_bs...; [state.σs[i] * partial_bs[j] for j=1:state.M for i=1:N]...])
    end
    error("unrecognized measurement!")
end

In [ ]:
# Gradient descent RBM learning for ground state of quantum Ising transverse field
g = 1.00
# exact energy of the peridoic ITF per bond (N → ∞)
itfexact(g) = -quadgk(k->sqrt(1+g^2-2*g*cos(k)), 0, pi)[1]/pi
@show itfexact(g) * N

# learning rate
λ = 0.01

n_updates = 100
for step = 1:n_updates
    N, M = state.N, state.M
    n_mcsweeps = 100
    l = 0
    O_av    = zeros(n_pars)
    E_av    = 0.0
    szsz_av = zeros(N)
    sx_av   = zeros(N)
    EO_av   = zeros(n_pars)
    n_samples = n_mcsweeps*N
    counter = 0
    while l < n_samples
        #print("x")
        spin = rand(1:N)
        counter += 1
        if propose_update!(state, spin)
            l += 1
            szsz     = measure(state, :ZZ)
            sx       = measure(state, :X)
            OStar    = measure(state, :OStar)
            E = -sum(szsz) - g*sum(sx)
            O_av += OStar
            E_av += E
            EO_av += E .* OStar
        end
        if counter > 20*n_samples
            error()
        end
    end

    EO_av /= n_samples
    O_av /= n_samples
    E_av /= n_samples
    F = EO_av - E_av .* O_av    
    println(E_av)

    # update weights and bias
    state.a -= λ *F[1:N]
    state.b -= λ *F[N+1:N+M]
    state.w -= λ *reshape(F[N+M+1:n_pars], N, M)

    recalc_θs!(state)
    recalc_amp!(state)
end
println()